In [ ]:
#Extract Youtube Thumbnails based on Channel ID

data_api_key = "" #insert Youtube API key
youtune_api_version ="v3"
youtube_api_service_name = "youtube"


In [ ]:
from googleapiclient.discovery import build
import json
import pandas as pd
import numpy as np
import os
import requests
import math
import time

In [ ]:
#Create folder based on timestamp to store pics

current_time = time.localtime()
record_timestamp =  str(current_time.tm_year) + "_" + str(current_time.tm_mon) + "_" + str(current_time.tm_mday)

folder_name = "images_ori_" + record_timestamp
path = os.getcwd()
try:
    os.mkdir(folder_name)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:
    print ("Successfully created the directory %s " % path)



In [ ]:
def extract_yt_info_urls(response_include, max_results, channel_id, by_order):
    """Extract Thumbnail URls based on Youtube Channel IDs"""
    
    result_counter = max_results
    my_token = None
    urls = []
    if max_results < 50:
        max_result_cycle = max_results
    else:
        max_result_cycle = 50
    
     
    for cycle in range(math.ceil(max_results / 50)):
        print(cycle)

        search_yt_res = service_yt.search().list(part = response_include, 
                                             maxResults = max_result_cycle,  
                                             channelId = channel_id,
                                             order = by_order,  
                                             pageToken = my_token
                                                 ).execute()

        
    
        for result in search_yt_res.get("items"):                                #Extract Urls
            url = str(result["snippet"]["thumbnails"]["high"]["url"])
            urls.append(url)
            
            dump = json.dumps(search_yt_res, sort_keys=True, indent=5)
            #print(dump)
            
            
        
        if (cycle + 1) <  (math.ceil(max_results / 50)):
            
            if search_yt_res["pageInfo"]["resultsPerPage"] < 50:
                break

            my_token = search_yt_res["nextPageToken"]
            time.sleep(1)
        
        
        result_counter = result_counter - max_result_cycle
        print(result_counter)
        if result_counter < 50:
            max_result_cycle = result_counter        
    
    
    return urls

def save_image_url(write_path,th_url):
    """Downloads pics based on URL"""
    with open(write_path, 'wb') as handler:
            response = requests.get(th_url , timeout = None,stream=True)   

            if not response.ok:
                print(response)

            for block in response.iter_content(2048):  
                if not block:
                    break

                handler.write(block)

In [ ]:
#Settings Extraction

max_results = 450   # per Youtube channel                              

channel_ids = ["",
               "",
               "",
               "",
               ""]

response_include = "id,snippet" 
sort_order = "date"   # Select order -> relevance, date



In [ ]:
#Connect to API, Extract Thumbnails URLs, Download Pictures and Save.
service_yt = build(youtube_api_service_name, youtune_api_version, developerKey = data_api_key)

for index_channel , channel in enumerate(channel_ids):
    t1=time.time()
    my_urls = extract_yt_info_urls(response_include = response_include , 
                                   max_results = max_results , 
                                   channel_id = channel , 
                                   by_order = sort_order
                                  )
    
    
    
    for index_image , url in enumerate(my_urls):
        name_file = str(index_channel + 1 ) + "_" + str(index_image + 1 )
        image_path = "{0}/{1}.jpg".format(folder_name , name_file)
        save_image_url(write_path = image_path,th_url =  url)
    
    
    t2=time.time()
    print((t2-t1))